# Diagnóstico y mejora de una consulta lenta
###Contexto:
Una consulta SQL ineficiente puede comprometer el rendimiento de toda una aplicación. Este ejercicio tiene como objetivo que los estudiantes apliquen buenas prácticas para optimizar una consulta.

###Consigna:
A partir de una consulta SQL poco optimizada, que será entregada por el docente, los estudiantes deben identificar los problemas presentes y proponer al menos tres mejoras justificadas, tales como: uso de índices, reducción de columnas seleccionadas, reestructuración de la consulta, entre otros.

###Paso a paso:

1- Lee y comprende la consulta entregada por el docente.

2- Analiza qué recursos podría estar consumiendo en exceso.

3- Aplica herramientas como EXPLAIN o EXPLAIN ANALYZE para visualizar el plan de ejecución.

4- Redacta una versión optimizada de la consulta y explica cada uno de los cambios realizados.

5- Comparte tus mejoras y justificaciones con el grupo.

In [6]:
import sqlite3
import pandas as pd

# Crear conexión a base de datos en memoria
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Crear tabla de clientes
cursor.execute('''
CREATE TABLE clientes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT,
    correo TEXT,
    edad INTEGER
)
''')

# Crear tabla de ordenes
cursor.execute('''
CREATE TABLE ordenes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cliente_id INTEGER,
    fecha TEXT,
    total REAL,
    FOREIGN KEY(cliente_id) REFERENCES clientes(id)
)
''')

# Insertar algunos datos (20 clientes y sus órdenes)
nombres = ['Carlos', 'Ana', 'Luis', 'María', 'Pedro', 'Sofía', 'Juan', 'Laura', 'Miguel', 'Camila',
           'Andrés', 'Paula', 'José', 'Valentina', 'Jorge', 'Elena', 'Ricardo', 'Martina', 'Tomás', 'Fernanda']

for i, nombre in enumerate(nombres, start=1):
    cursor.execute("INSERT INTO clientes (nombre, correo, edad) VALUES (?, ?, ?)",
                   (nombre, f'{nombre.lower()}@mail.com', 20 + (i % 25)))
    cursor.execute("INSERT INTO ordenes (cliente_id, fecha, total) VALUES (?, DATE('now', ? || ' days'), ?)",
                   (i, f'-{i}', i * 10.0))

conn.commit()

# Consulta lenta (no optimizada)
consulta_lenta = '''
SELECT *
FROM clientes c
JOIN ordenes o ON c.id = o.cliente_id
WHERE c.edad > 30
ORDER BY o.fecha DESC
'''

print("🔍 Resultados de la consulta lenta:")
display(pd.read_sql_query(consulta_lenta, conn))

# Ver plan de ejecución
print("🛠 Plan de ejecución de la consulta lenta:")
plan = cursor.execute("EXPLAIN QUERY PLAN " + consulta_lenta)
for row in plan:
    print(row)

# Crear índices
cursor.execute("CREATE INDEX idx_edad ON clientes(edad)")
cursor.execute("CREATE INDEX idx_cliente_id ON ordenes(cliente_id)")
conn.commit()

# Consulta optimizada
consulta_opt = '''
SELECT c.nombre, c.edad, o.fecha, o.total
FROM clientes c
JOIN ordenes o ON c.id = o.cliente_id
WHERE c.edad > 30
ORDER BY o.fecha DESC
LIMIT 10
'''

print("\n🚀 Resultados de la consulta optimizada:")
display(pd.read_sql_query(consulta_opt, conn))


🔍 Resultados de la consulta lenta:


,id,nombre,correo,edad,id,cliente_id,fecha,total
0,11,Andrés,andrés@mail.com,31,11,11,2025-07-19,110.0
1,12,Paula,paula@mail.com,32,12,12,2025-07-18,120.0
2,13,José,josé@mail.com,33,13,13,2025-07-17,130.0
3,14,Valentina,valentina@mail.com,34,14,14,2025-07-16,140.0
4,15,Jorge,jorge@mail.com,35,15,15,2025-07-15,150.0
5,16,Elena,elena@mail.com,36,16,16,2025-07-14,160.0
6,17,Ricardo,ricardo@mail.com,37,17,17,2025-07-13,170.0
7,18,Martina,martina@mail.com,38,18,18,2025-07-12,180.0
8,19,Tomás,tomás@mail.com,39,19,19,2025-07-11,190.0
9,20,Fernanda,fernanda@mail.com,40,20,20,2025-07-10,200.0


🛠 Plan de ejecución de la consulta lenta:
(4, 0, 0, 'SCAN o')
(6, 0, 0, 'SEARCH c USING INTEGER PRIMARY KEY (rowid=?)')
(23, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')

🚀 Resultados de la consulta optimizada:


,nombre,edad,fecha,total
0,Andrés,31,2025-07-19,110.0
1,Paula,32,2025-07-18,120.0
2,José,33,2025-07-17,130.0
3,Valentina,34,2025-07-16,140.0
4,Jorge,35,2025-07-15,150.0
5,Elena,36,2025-07-14,160.0
6,Ricardo,37,2025-07-13,170.0
7,Martina,38,2025-07-12,180.0
8,Tomás,39,2025-07-11,190.0
9,Fernanda,40,2025-07-10,200.0


# Diseño de un esquema eficiente con vistas e índices
### Contexto:
El diseño del esquema de base de datos afecta directamente el rendimiento de las consultas, especialmente en aplicaciones que manejan grandes volúmenes de información.

### Consigna:
Diseña una estructura de base de datos simple para una aplicación de eCommerce que permita consultas eficientes sobre ventas, productos y usuarios.
Incluye al menos una vista y dos índices justificados.

### Paso a paso:
Piensa qué entidades necesita tu esquema, por ejemplo: usuarios, productos, órdenes, etc.

1- Define las claves primarias y foráneas necesarias para relacionar correctamente las tablas.

2- Elige qué columnas indexar para mejorar el rendimiento en las consultas típicas (por ejemplo: por fecha, por usuario, por categoría de producto, etc.).

3- Diseña una vista que facilite el análisis de ventas por mes.

4- Justifica por qué tu diseño mejora el rendimiento de las consultas en la base de datos.

In [8]:
import sqlite3
import pandas as pd

# Crear conexión en memoria
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Crear tabla de usuarios
cursor.execute('''
CREATE TABLE usuarios (
    id_usuario INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    fecha_registro DATE
)
''')

# Crear tabla de productos
cursor.execute('''
CREATE TABLE productos (
    id_producto INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    categoria TEXT,
    precio DECIMAL(10, 2)
)
''')

# Crear tabla de órdenes
cursor.execute('''
CREATE TABLE ordenes (
    id_orden INTEGER PRIMARY KEY,
    id_usuario INTEGER,
    id_producto INTEGER,
    fecha DATE,
    cantidad INTEGER,
    total DECIMAL(10, 2),
    FOREIGN KEY (id_usuario) REFERENCES usuarios(id_usuario),
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto)
)
''')

# Insertar datos de prueba
usuarios_data = [
    (1, 'Lucía Ríos', 'lucia@example.com', '2024-01-15'),
    (2, 'Tomás Vidal', 'tomas@example.com', '2024-02-20')
]
productos_data = [
    (1, 'Teclado Gamer', 'Electrónica', 39990),
    (2, 'Mouse Inalámbrico', 'Electrónica', 15990)
]
ordenes_data = [
    (1, 1, 1, '2024-06-01', 1, 39990),
    (2, 2, 2, '2024-06-05', 2, 31980),
    (3, 1, 2, '2024-07-01', 1, 15990)
]

cursor.executemany('INSERT INTO usuarios VALUES (?, ?, ?, ?)', usuarios_data)
cursor.executemany('INSERT INTO productos VALUES (?, ?, ?, ?)', productos_data)
cursor.executemany('INSERT INTO ordenes VALUES (?, ?, ?, ?, ?, ?)', ordenes_data)

# Crear índices para mejorar rendimiento
cursor.execute('CREATE INDEX idx_ordenes_usuario ON ordenes(id_usuario)')
cursor.execute('CREATE INDEX idx_ordenes_fecha ON ordenes(fecha)')

# Crear vista de ventas mensuales
cursor.execute('''
CREATE VIEW ventas_mensuales AS
SELECT
    STRFTIME('%Y-%m', fecha) AS mes,
    SUM(total) AS total_ventas,
    COUNT(*) AS cantidad_ordenes
FROM ordenes
GROUP BY mes
ORDER BY mes
''')

# Consultar la vista
print("📊 Ventas mensuales:")
display(pd.read_sql_query('SELECT * FROM ventas_mensuales', conn))

# Cerrar conexión (opcional)
# conn.close()


📊 Ventas mensuales:


,mes,total_ventas,cantidad_ordenes
0,2024-06,71970,2
1,2024-07,15990,1
